## This code is developed in Julia 1.5.3 version. One may require to do some minor modifications in order to run the code in a recent version of Julia.

In [ ]:
using  GridapGmsh
using  Gridap
using  Gridap.Geometry
using  Gridap.TensorValues
using  Plots

In [ ]:
const T0 = 300
const TAppMax = T0 + 50
const delt = 1e-2
const tMax = 1
const uMax = 1.2e-6
AppVel = uMax/tMax
uMin = 0
uTran = 0.2*uMax

In [ ]:
using SymPy
x,x₁ = symbols("x,x₁", real = true)
heaviside(x) = 0.5 * (sign(x) + 1)
interval(x, a, b) = heaviside(x-a) - heaviside(x-b)
hS = uMax/10
F(x) = (T0 - TAppMax) * interval(x,-4*hS + uMin,uTran)
wₕ(x,x₁) = (1/(sqrt(2*pi)*hS))*exp(-(x-x₁)^2/(2*hS^2))
smoothT = SymPy.integrate(F(x₁)*wₕ(x,x₁),(x₁,-4*hS,uMax)) +  TAppMax
plot(smoothT,0,uMax)

In [ ]:
function Tfun(u)  
    if u <= uTran
      return ((TAppMax - T0)/uTran)*u + T0
    else
     return  TAppMax
    end
end 
plot(Tfun,0,uMax)

In [ ]:
uAppVec = range(0,uMax,length = Int64(floor(tMax/delt)))

AppTOption = 1 ## 1 for smooth and otherwise linear than constant
if AppTOption == 1
    TAppVec = smoothT.(uAppVec)
  else
    TAppVec = Tfun.(uAppVec) 
end 

In [ ]:
const E = 340.0e9
const ν= 0.22
const Gc = 42.47
const η= 1e-8
const α= 8e-6
const c = 0.775
const κ_mat = 300.0
const ρ= 2450

In [ ]:
using gmsh

const L = 1e-3
const H = 1e-3
const lsp = L/300
const CP = H/2
const CL = 0.5*L 
const CH = H/2000
const hfc = lsp/5
const hf = lsp/4
const h = 20*hf 
const FMR = 6*lsp
 
gmsh.initialize()
gmsh.option.setNumber("General.Terminal", 1)
gmsh.model.geo.addPoint(0.0, 0.0, 0.0, h, 1)  
gmsh.model.geo.addPoint(L, 0.0, 0.0, h, 2) 
gmsh.model.geo.addPoint(L, H, 0.0, h, 3) 
gmsh.model.geo.addPoint(0.0, H, 0.0, h, 4)
gmsh.model.geo.addPoint(0.0, CP + CH, 0.0, h, 5)
gmsh.model.geo.addPoint(CL, CP + CH, 0.0, hfc, 6)
gmsh.model.geo.addPoint(CL, CP - CH, 0.0, hfc, 7)
gmsh.model.geo.addPoint(0.0, CP - CH, 0.0, h, 8)

gmsh.model.geo.addLine(1, 2, 1)
gmsh.model.geo.addLine(2, 3, 2)
gmsh.model.geo.addLine(3, 4, 3)
gmsh.model.geo.addLine(4, 5, 4)
gmsh.model.geo.addLine(5, 6, 5)
gmsh.model.geo.addLine(6, 7, 6)
gmsh.model.geo.addLine(7, 8, 7)
gmsh.model.geo.addLine(8, 1, 8)
gmsh.model.geo.addCurveLoop([1,2,3,4,5,6,7,8],1) 
gmsh.model.geo.addPlaneSurface([1], 1)
gmsh.model.addPhysicalGroup(2, [1],1)
gmsh.model.addPhysicalGroup(1, [1],1)
gmsh.model.addPhysicalGroup(1, [3],2)
gmsh.model.setPhysicalName(2, 1, "Domain")
gmsh.model.setPhysicalName(1, 1, "BottomEdge")
gmsh.model.setPhysicalName(1, 2, "TopEdge")


gmsh.model.mesh.field.add("Box", 11)
gmsh.model.mesh.field.setNumber(11, "VIn", hf)
gmsh.model.mesh.field.setNumber(11, "VOut", h)
gmsh.model.mesh.field.setNumber(11, "XMin", 0.8*CL)
gmsh.model.mesh.field.setNumber(11, "XMax", L)
gmsh.model.mesh.field.setNumber(11, "YMin", CP-FMR)
gmsh.model.mesh.field.setNumber(11, "YMax", CP+FMR)
gmsh.model.mesh.field.setAsBackgroundMesh(11)

gmsh.model.geo.synchronize()
gmsh.model.mesh.generate(2)
gmsh.write("SquarePlateWithEdgeNotch.msh")
gmsh.finalize()

In [ ]:
model = GmshDiscreteModel("SquarePlateWithEdgeNotch.msh")
writevtk(model ,"SquarePlateWithEdgeNotch")

In [ ]:
function  ElasFourthOrderConstTensor(E,ν,PlanarState)# 1 for  Plane  Stress  and 2 Plane  Strain  Condition
    if  PlanarState  == 1
        C1111 =E/(1-ν*ν)
        C1122 = (ν*E)/(1-ν*ν)
        C1112 = 0.0
        C2222 =E/(1-ν*ν)
        C2212 = 0.0
        C1212 =E/(2*(1+ν))
    elseif  PlanarState  == 2
        C1111 = (E*(1-ν*ν))/((1+ν)*(1-ν-2*ν*ν))
        C1122 = (ν*E)/(1-ν-2*ν*ν)
        C1112 = 0.0
        C2222 = (E*(1-ν))/(1-ν-2*ν*ν)
        C2212 = 0.0
        C1212 =E/(2*(1+ν))
    end
    C_ten = SymFourthOrderTensorValue(C1111 ,C1112 ,C1122 ,C1112 ,
        C1212 ,C2212 ,C1122 ,C2212 ,C2222)
    return   C_ten
end

In [ ]:
const  C_mat = ElasFourthOrderConstTensor(E,ν,2)

In [ ]:
I2 = SymTensorValue{2,Float64}(1.0 ,0.0 ,1.0)
I4 = I2 ⊗ I2
I4_sym = one(SymFourthOrderTensorValue{2,Float64})
I4_vol = (1.0/3)*I4
I4_dev = I4_sym  - I4_vol

In [ ]:
σ_elas(εElas) = C_mat ⊙ εElas

In [ ]:
function σ_elasMod(ε,ε_in , s_in ,T,T_in)
    εElas_in = ε_in - α*(T_in -T0)*I2
    εElas = ε - α*(T-T0)*I2
    if tr(εElas_in)  >= 0
        σ= (s_in^2 +η)*σ_elas(εElas)
    elseif  tr(εElas_in) < 0
        σ= (s_in^2 +η) *I4_dev ⊙ σ_elas(εElas) + I4_vol ⊙ σ_elas(εElas)
    end
    return σ
end

In [ ]:
function σ_totMod(ε,ε_in ,s_in ,T,T_in)
    εElas_in = ε_in - α*(T_in -T0)*I2
    εElasTot = ε - α*T*I2
    if tr(εElas_in) >= 0
        σT = (s_in^2 +η)*σ_elas(εElasTot)
    elseif  tr(εElas_in) < 0
        σT = (s_in^2 +η) *I4_dev ⊙ σ_elas(εElasTot) + I4_vol ⊙ σ_elas(εElasTot)
    end
    return σT
end

In [ ]:
function σ_thermMod(ε_in ,s_in ,T_in)
    εElas_in = ε_in - α*(T_in -T0)*I2
    εElasTher = α*(T0)*I2
    if tr(εElas_in) >= 0
        σF = (s_in^2 +η)*σ_elas(εElasTher)
    elseif  tr(εElas_in) < 0
        σF = (s_in^2 +η)*I4_dev ⊙ σ_elas(εElasTher) + I4_vol ⊙ σ_elas(εElasTher)
    end
    return σF
end

In [ ]:
function ΨPos(ε_in ,T_in)
    εElas_in = ε_in - α*(T_in -T0)*I2
    if tr(εElas_in)  >= 0
        ΨPlus = 0.5*((εElas_in) ⊙ σ_elas(εElas_in))
    elseif  tr(εElas_in) < 0
        ΨPlus = 0.5*(( I4_dev ⊙ σ_elas(εElas_in)) ⊙ (I4_dev ⊙ (εElas_in)))
    end
    return ΨPlus
end

In [ ]:
order = 1
reffe_PF = ReferenceFE(lagrangian ,Float64 ,order)
V0_PF = TestFESpace(model ,reffe_PF;conformity =:H1)
U_PF = TrialFESpace(V0_PF)
sh = zero(V0_PF)

In [ ]:
reffe_Disp = ReferenceFE(lagrangian ,VectorValue{2,Float64},order)
V0_Disp = TestFESpace(model ,reffe_Disp;conformity =:H1 ,dirichlet_tags =["BottomEdge","TopEdge"],
    dirichlet_masks =[(true ,true) ,(false ,true)])
uh = zero(V0_Disp)

In [ ]:
reffe_Temp = ReferenceFE(lagrangian ,Float64 ,order)
V0_Temp = TestFESpace(model ,reffe_Temp;
    conformity =:H1 ,dirichlet_tags =["BottomEdge","TopEdge"])

In [ ]:
V0 = MultiFieldFESpace([V0_Disp,V0_Temp])

In [ ]:
degree = 2* order
Ω= Triangulation(model)
dΩ= Measure(Ω,degree)

In [ ]:
labels = get_face_labeling(model)
LoadTagId = get_tag_from_name(labels ,"TopEdge")
Γ_Load = BoundaryTriangulation(model ,tags = LoadTagId)
dΓ_Load = Measure(Γ_Load ,degree)
n_Γ_Load = get_normal_vector(Γ_Load)

In [ ]:
function  project(q,model ,dΩ,order)
    reffe = ReferenceFE(lagrangian ,Float64 ,order)
    V = FESpace(model ,reffe ,conformity =:L2)
    a(u,v) =∫(u*v)*dΩ
    b(v) =∫(v*q)*dΩ
    op = AffineFEOperator(a,b,V,V)
    qh = Gridap.solve(op)
    return  qh
end

In [ ]:
function stepPhaseField(uh_in ,ΨPlusPrev_in)
    a_PF(s,Φ) =∫( Gc*lsp*∇(Φ)⋅ ∇(s) + 2*ΨPlusPrev_in*s*Φ
        + (Gc/lsp)*s*Φ)*dΩ
    b_PF(Φ) =∫( (Gc/lsp)*Φ)*dΩ
    op_PF = AffineFEOperator(a_PF ,b_PF ,U_PF ,V0_PF)
    sh_out = Gridap.solve(op_PF)
    return  sh_out
end

In [ ]:
κGradTemp(∇,s_in) = (s_in^2 +η)*κ_mat*∇

In [ ]:
function    stepDispTemp(uh_in ,sh_in ,T_in ,vApp ,TApp ,delt)
    uApp1(x) = VectorValue(0.0,0.0)
    uApp2(x) = VectorValue(0.0,vApp)
    U_Disp = TrialFESpace(V0_Disp ,[uApp1 ,uApp2])
    Tapp1(x) = T0
    Tapp2(x) = TApp
    Tg = TrialFESpace(V0_Temp ,[Tapp1 , Tapp2])
    U = MultiFieldFESpace([U_Disp ,Tg])
    a((u,T) ,(v,w)) =∫( (ε(v) ⊙ (σ_totMod∘(ε(u),ε(uh_in),sh_in ,T,T_in))) + ∇(w)⋅(κGradTemp∘(∇(T),sh_in)) + ((ρ*c*T*w)/delt))*dΩ
    b((v,w)) =∫(((ρ*c*T_in*w)/delt) - (ε(v)⊙(σ_thermMod∘(ε(uh_in),sh_in ,T_in))))*dΩ
    op = AffineFEOperator(a,b,U,V0)
    uhTh = Gridap.solve(op)
    uh_out ,Th_out =   uhTh
    return  uh_out ,Th_out
end

In [ ]:
function  new_EnergyState(ψPlusPrev_in ,ψhPos_in)
    ψPlus_in = ψhPos_in
    if ψPlus_in  >= ψPlusPrev_in
        ψPlus_out = ψPlus_in
    else
        ψPlus_out =ψPlusPrev_in
    end
    true ,ψPlus_out
end

In [ ]:
t = 0.0
innerMax = 10
count = 0
tol = 1e-8
Load = Float64[]
Displacement = Float64[]
push!(Load , 0.0)
push!(Displacement , 0.0)
ΨPlusPrev = CellState(0.0,dΩ)
sPrev = CellState(1.0,dΩ)
sh = project(sPrev ,model ,dΩ,order)
ThPrev = CellState(T0,dΩ)
Th = project(ThPrev ,model ,dΩ,order)
while t .< tMax
    count = count  .+ 1
    t = t + delt
    vApp = AppVel*t
    TApp = TAppVec[count]
    print("\n Entering  displacemtent  step :", float(vApp))
    for  inner = 1:innerMax
        ΨhPlusPrev = project(ΨPlusPrev ,model ,dΩ,order)
        RelErr = abs(sum(∫( Gc*lsp*∇(sh)⋅ ∇(sh) + 2*ΨhPlusPrev*sh*sh + (Gc/lsp)*sh*sh)*dΩ-∫( (Gc/lsp)*sh)*dΩ))/abs(sum(∫( (Gc/lsp)*sh)*dΩ))
        sh = stepPhaseField(uh,ΨhPlusPrev)
        uh ,Th = stepDispTemp(uh,sh ,Th,vApp ,TApp ,delt)
        ΨhPos_in =ΨPos∘(ε(uh),Th)
        update_state!( new_EnergyState ,ΨPlusPrev ,ΨhPos_in)
        if  RelErr  < tol
            break
        end
    end
    Node_Force = sum(∫( n_Γ_Load⋅(σ_elasMod∘(ε(uh),ε(uh),sh,Th,Th)) ) *dΓ_Load)
    push!(Load , Node_Force[2])
    push!( Displacement , vApp)
    writevtk(Ω,"results_PhaseFieldThermoElastic",cellfields=["uh"=>uh,"s"=>sh ,"epsi"=>ε(uh),"T"=>Th])
end

In [ ]:
plot(Displacement ,Load)